In [1]:
""" Apply industry weights on merged table.
-------------------------------------------------------------------------------

This script applies the industry weights to the framework. Overall Water Risk
(OWR) is calculated for every industry. When scores are unavailable (nan),
the weights have been set to Nan to exclude them from the weight sum. 

Todo:
overall water risk scores per category (e.g.) have not been calculated yet. 


Author: Rutger Hofste
Date: 20181211
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""

SCRIPT_NAME = 'Y2018M12D11_RH_Master_Weights_GPD_V01'
OUTPUT_VERSION = 3

BQ_IN = {}
# Master Table
BQ_IN["MASTER"] = "y2018m12d04_rh_master_merge_rawdata_gpd_v01_v02"

# Weights
BQ_IN["WEIGHTS"] ="y2018m12d06_rh_process_weights_bq_v01_v01"

BQ_PROJECT_ID = "aqueduct30"
BQ_OUTPUT_DATASET_NAME = "aqueduct30v01"
BQ_OUTPUT_TABLE_NAME = "{}_v{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION).lower()

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("\nBQ_OUTPUT_DATASET_NAME: ", BQ_OUTPUT_DATASET_NAME,
      "\nBQ_OUTPUT_TABLE_NAME: ", BQ_OUTPUT_TABLE_NAME,
      "\ns3_output_path: ", s3_output_path,
      "\nec2_output_path:" , ec2_output_path)


BQ_OUTPUT_DATASET_NAME:  aqueduct30v01 
BQ_OUTPUT_TABLE_NAME:  y2018m12d11_rh_master_weights_gpd_v01_v03 
s3_output_path:  s3://wri-projects/Aqueduct30/processData/Y2018M12D11_RH_Master_Weights_GPD_V01/output_V03/ 
ec2_output_path: /volumes/data/Y2018M12D11_RH_Master_Weights_GPD_V01/output_V03


In [2]:
BQ_IN

{'MASTER': 'y2018m12d04_rh_master_merge_rawdata_gpd_v01_v02',
 'WEIGHTS': 'y2018m12d06_rh_process_weights_bq_v01_v01'}

In [3]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M12D13 UTC 16:51


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [4]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

In [5]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/.google.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "aqueduct30"
client = bigquery.Client(project=BQ_PROJECT_ID)

pd.set_option('display.max_columns', 500)

In [6]:
sql_master = """
SELECT
  *
FROM
  `{}.{}.{}`
ORDER BY
  aq30_id
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["MASTER"])

In [7]:
df_master = pd.read_gbq(query=sql_master,dialect="standard")

In [8]:
df_master.head()

,aq30_id,string_id,pfaf_id,gid_1,aqid,area_km2,name_1,gid_0,name_0,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1,temporal_resolution,delta_id,year,bws_raw,bws_score,bws_cat,bws_label,bwd_raw,bwd_score,bwd_cat,bwd_label,iav_raw,iav_score,iav_cat,iav_label,sev_raw,sev_score,sev_cat,sev_label,rfr_raw,rfr_score,rfr_cat,rfr_label,cfr_raw,cfr_score,cfr_cat,cfr_label,drr_raw,drr_score,drr_cat,drr_label,gtd_raw,gtd_score,gtd_cat,gtd_label,ucw_raw,ucw_score,ucw_cat,ucw_label,cep_raw,cep_score,cep_cat,cep_label,udw_raw,udw_score,udw_cat,udw_label,usa_raw,usa_score,usa_cat,usa_label,rri_raw,rri_score,rri_cat,rri_label
0,0,111011-EGY.11_1-3365,111011.0,EGY.11_1,3365.0,4.223754,Al Qahirah,EGY,Egypt,Cairo|El Cairo|El Qahira|Le Caire,None,Muhafazah,Governorate,None,EG.QH,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,0.843958,NaN,NaN,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),58.0,2.8,2.0,Medium to high (50 to 60%)
1,1,111011-EGY.15_1-3365,111011.0,EGY.15_1,3365.0,1846.012343,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,0.843958,NaN,NaN,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),58.0,2.8,2.0,Medium to high (50 to 60%)
2,2,111011-EGY.15_1-None,111011.0,EGY.15_1,NaN,30.526067,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,NaN,NaN,NaN,None,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),58.0,2.8,2.0,Medium to high (50 to 60%)
3,3,111011-None-3365,111011.0,None,3365.0,0.742712,None,None,None,None,None,None,None,None,None,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,0.843958,NaN,NaN,Insignificant Trend,NaN,NaN,NaN,None,0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),NaN,NaN,NaN,None
4,4,111011-None-None,111011.0,None,NaN,13.430995,None,None,None,None,None,None,None,None,None,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),NaN,NaN,NaN,None


In [9]:
df_master.shape

(68511, 70)

In [10]:
sql_weights = """
SELECT
  id,
  category_full_name,
  category_short,
  indicator_name_full,
  indicator_short,
  industry_full,
  industry_short,
  weight_abs,
  weight_label,
  weight_interpretation,
  weight_fraction
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["WEIGHTS"])

In [11]:
df_weights = pd.read_gbq(query=sql_weights,dialect="standard")

In [12]:
df_weights.head()

,id,category_full_name,category_short,indicator_name_full,indicator_short,industry_full,industry_short,weight_abs,weight_label,weight_interpretation,weight_fraction
0,88,Physical Risk Quality,QAL,Coastal eutrophication potential,CEP,Oil & Gas,ONG,0.0,No weight,Not relevant,0.000000
1,9,Physical Risk Quality,QAL,Untreated collected wastewater,UCW,Default,DEF,2.0,High,Represents high risk to the industry,0.081633
2,35,Physical Risk Quality,QAL,Untreated collected wastewater,UCW,Food & Beverage,FNB,2.0,High,Represents high risk to the industry,0.081633
3,36,Physical Risk Quality,QAL,Coastal eutrophication potential,CEP,Food & Beverage,FNB,2.0,High,Represents high risk to the industry,0.081633
4,48,Physical Risk Quality,QAL,Untreated collected wastewater,UCW,Chemicals,CHE,2.0,High,Represents high risk to the industry,0.076190


In [13]:
df_weights.shape

(130, 11)

In [14]:
industries = list(df_weights.industry_short.unique())
categories = list(df_weights.category_short.unique())
indicators = list(df_weights.indicator_short.unique())



In [15]:
# Calculate Overall Water Risks by using weights

In [16]:
df_merged_weights = df_master.copy()

In [17]:
for industry in industries:
    for indicator in indicators:
        column_name_weight = "{}_{}_weight".format(indicator.lower(),industry.lower())
        column_name_score = "{}_score".format(indicator.lower())
        column_name_weighted_score = "{}_{}_weightedscore".format(indicator.lower(),industry.lower())
        weight = df_weights.loc[(df_weights.industry_short == industry) & (df_weights.indicator_short == indicator)].weight_fraction.iloc[0]
        score = df_master[column_name_score]
               
        df_merged_weights[column_name_weight] = weight
        df_merged_weights[column_name_weighted_score] = score * weight

In [18]:
df_merged_weights.head()

,aq30_id,string_id,pfaf_id,gid_1,aqid,area_km2,name_1,gid_0,name_0,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1,temporal_resolution,delta_id,year,bws_raw,bws_score,bws_cat,bws_label,bwd_raw,bwd_score,bwd_cat,bwd_label,iav_raw,iav_score,iav_cat,iav_label,sev_raw,sev_score,sev_cat,sev_label,rfr_raw,rfr_score,rfr_cat,rfr_label,cfr_raw,cfr_score,cfr_cat,cfr_label,drr_raw,drr_score,drr_cat,drr_label,gtd_raw,gtd_score,gtd_cat,gtd_label,ucw_raw,ucw_score,ucw_cat,ucw_label,cep_raw,cep_score,cep_cat,cep_label,udw_raw,udw_score,udw_cat,udw_label,usa_raw,usa_score,usa_cat,usa_label,rri_raw,rri_score,rri_cat,rri_label,cep_ong_weight,cep_ong_weightedscore,ucw_ong_weight,ucw_ong_weightedscore,drr_ong_weight,drr_ong_weightedscore,iav_ong_weight,iav_ong_weightedscore,bws_ong_weight,bws_ong_weightedscore,bwd_ong_weight,bwd_ong_weightedscore,gtd_ong_weight,gtd_ong_weightedscore,sev_ong_weight,sev_ong_weightedscore,rfr_ong_weight,rfr_ong_weightedscore,cfr_ong_weight,cfr_ong_weightedscore,udw_ong_weight,udw_ong_weightedscore,usa_ong_weight,usa_ong_weightedscore,rri_ong_weight,rri_ong_weightedscore,cep_def_weight,cep_def_weightedscore,ucw_def_weight,ucw_def_weightedscore,drr_def_weight,drr_def_weightedscore,iav_def_weight,iav_def_weightedscore,bws_def_weight,bws_def_weightedscore,bwd_def_weight,bwd_def_weightedscore,gtd_def_weight,gtd_def_weightedscore,sev_def_weight,sev_def_weightedscore,rfr_def_weight,rfr_def_weightedscore,cfr_def_weight,cfr_def_weightedscore,udw_def_weight,udw_def_weightedscore,usa_def_weight,usa_def_weightedscore,rri_def_weight,rri_def_weightedscore,cep_fnb_weight,cep_fnb_weightedscore,ucw_fnb_weight,ucw_fnb_weightedscore,drr_fnb_weight,drr_fnb_weightedscore,iav_fnb_weight,iav_fnb_weightedscore,bws_fnb_weight,bws_fnb_weightedscore,bwd_fnb_weight,bwd_fnb_weightedscore,gtd_fnb_weight,gtd_fnb_weightedscore,sev_fnb_weight,sev_fnb_weightedscore,rfr_fnb_weight,rfr_fnb_weightedscore,cfr_fnb_weight,cfr_fnb_weightedscore,udw_fnb_weight,udw_fnb_weightedscore,usa_fnb_weight,usa_fnb_weightedscore,rri_fnb_weight,rri_fnb_weightedscore,cep_che_weight,cep_che_weightedscore,ucw_che_weight,ucw_che_weightedscore,drr_che_weight,drr_che_weightedscore,iav_che_weight,iav_che_weightedscore,bws_che_weight,bws_che_weightedscore,bwd_che_weight,bwd_che_weightedscore,gtd_che_weight,gtd_che_weightedscore,sev_che_weight,sev_che_weightedscore,rfr_che_weight,rfr_che_weightedscore,cfr_che_weight,cfr_che_weightedscore,udw_che_weight,udw_che_weightedscore,usa_che_weight,usa_che_weightedscore,rri_che_weight,rri_che_weightedscore,cep_smc_weight,cep_smc_weightedscore,ucw_smc_weight,ucw_smc_weightedscore,drr_smc_weight,drr_smc_weightedscore,iav_smc_weight,iav_smc_weightedscore,bws_smc_weight,bws_smc_weightedscore,bwd_smc_weight,bwd_smc_weightedscore,gtd_smc_weight,gtd_smc_weightedscore,sev_smc_weight,sev_smc_weightedscore,rfr_smc_weight,rfr_smc_weightedscore,cfr_smc_weight,cfr_smc_weightedscore,udw_smc_weight,udw_smc_weightedscore,usa_smc_weight,usa_smc_weightedscore,rri_smc_weight,rri_smc_weightedscore,cep_agr_weight,cep_agr_weightedscore,ucw_agr_weight,ucw_agr_weightedscore,drr_agr_weight,drr_agr_weightedscore,iav_agr_weight,iav_agr_weightedscore,bws_agr_weight,bws_agr_weightedscore,bwd_agr_weight,bwd_agr_weightedscore,gtd_agr_weight,gtd_agr_weightedscore,sev_agr_weight,sev_agr_weightedscore,rfr_agr_weight,rfr_agr_weightedscore,cfr_agr_weight,cfr_agr_weightedscore,udw_agr_weight,udw_agr_weightedscore,usa_agr_weight,usa_agr_weightedscore,rri_agr_weight,rri_agr_weightedscore,cep_tex_weight,cep_tex_weightedscore,ucw_tex_weight,ucw_tex_weightedscore,drr_tex_weight,drr_tex_weightedscore,iav_tex_weight,iav_tex_weightedscore,bws_tex_weight,bws_tex_weightedscore,bwd_tex_weight,bwd_tex_weightedscore,gtd_tex_weight,gtd_tex_weightedscore,sev_tex_weight,sev_tex_weightedscore,rfr_tex_weight,rfr_tex_weightedscore,cfr_tex_weight,cfr_tex_weightedscore,udw_tex_weight,udw_tex_weightedscore,usa_tex_weight,usa_tex_weightedscore,rri_tex_weight,rri_tex_weightedscore,cep_elp_weight,

In [19]:
def mask_weights():
    """
    Sets weights to np.nan when the score is np.nan
    
    this is required for later steps in which sums are 
    calculated.
        
    """

    for industry in industries:
        for indicator in indicators:
            weight_column_name = "{}_{}_weight".format(indicator.lower(),industry.lower())
            score_column_name = "{}_{}_weightedscore".format(indicator.lower(),industry.lower())
            df_merged_weights[weight_column_name] = df_merged_weights[weight_column_name].mask(np.isnan(df_merged_weights[score_column_name]))

    return 1

mask_weights()

1

In [20]:
df_master.head()

,aq30_id,string_id,pfaf_id,gid_1,aqid,area_km2,name_1,gid_0,name_0,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1,temporal_resolution,delta_id,year,bws_raw,bws_score,bws_cat,bws_label,bwd_raw,bwd_score,bwd_cat,bwd_label,iav_raw,iav_score,iav_cat,iav_label,sev_raw,sev_score,sev_cat,sev_label,rfr_raw,rfr_score,rfr_cat,rfr_label,cfr_raw,cfr_score,cfr_cat,cfr_label,drr_raw,drr_score,drr_cat,drr_label,gtd_raw,gtd_score,gtd_cat,gtd_label,ucw_raw,ucw_score,ucw_cat,ucw_label,cep_raw,cep_score,cep_cat,cep_label,udw_raw,udw_score,udw_cat,udw_label,usa_raw,usa_score,usa_cat,usa_label,rri_raw,rri_score,rri_cat,rri_label
0,0,111011-EGY.11_1-3365,111011.0,EGY.11_1,3365.0,4.223754,Al Qahirah,EGY,Egypt,Cairo|El Cairo|El Qahira|Le Caire,None,Muhafazah,Governorate,None,EG.QH,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,0.843958,NaN,NaN,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),58.0,2.8,2.0,Medium to high (50 to 60%)
1,1,111011-EGY.15_1-3365,111011.0,EGY.15_1,3365.0,1846.012343,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,0.843958,NaN,NaN,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),58.0,2.8,2.0,Medium to high (50 to 60%)
2,2,111011-EGY.15_1-None,111011.0,EGY.15_1,NaN,30.526067,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,NaN,NaN,NaN,None,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),58.0,2.8,2.0,Medium to high (50 to 60%)
3,3,111011-None-3365,111011.0,None,3365.0,0.742712,None,None,None,None,None,None,None,None,None,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,0.843958,NaN,NaN,Insignificant Trend,NaN,NaN,NaN,None,0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),NaN,NaN,NaN,None
4,4,111011-None-None,111011.0,None,NaN,13.430995,None,None,None,None,None,None,None,None,None,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None,0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),NaN,NaN,NaN,None


In [21]:
for industry in industries:    
    # weights    
    regex_w = '^.*_{}_weight$|^string_id$'.format(industry.lower())
    df_w = df_merged_weights.filter(regex=regex_w)
    df_w = df_w.set_index("string_id")
    column_name_w = "{}_weight_sum".format(industry.lower())
    df_merged_weights[column_name_w] = df_w.sum(axis=1).values
        
    # Overall scores
    regex_s = '^.*_{}_weightedscore$|^string_id$'.format(industry.lower())
    df_s = df_merged_weights.filter(regex=regex_s)
    df_s = df_s.set_index("string_id")
    column_name_s = "{}_weightedscore_sum".format(industry.lower())
    df_merged_weights[column_name_s] = df_s.sum(axis=1).values
    df_merged_weights["owr_{}_raw".format(industry.lower())] = df_merged_weights[column_name_s] / df_merged_weights[column_name_w]

    
    

In [22]:
df_merged_weights.head()

,aq30_id,string_id,pfaf_id,gid_1,aqid,area_km2,name_1,gid_0,name_0,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1,temporal_resolution,delta_id,year,bws_raw,bws_score,bws_cat,bws_label,bwd_raw,bwd_score,bwd_cat,bwd_label,iav_raw,iav_score,iav_cat,iav_label,sev_raw,sev_score,sev_cat,sev_label,rfr_raw,rfr_score,rfr_cat,rfr_label,cfr_raw,cfr_score,cfr_cat,cfr_label,drr_raw,drr_score,drr_cat,drr_label,gtd_raw,gtd_score,gtd_cat,gtd_label,ucw_raw,ucw_score,ucw_cat,ucw_label,cep_raw,cep_score,cep_cat,cep_label,udw_raw,udw_score,udw_cat,udw_label,usa_raw,usa_score,usa_cat,usa_label,rri_raw,rri_score,rri_cat,rri_label,cep_ong_weight,cep_ong_weightedscore,ucw_ong_weight,ucw_ong_weightedscore,drr_ong_weight,drr_ong_weightedscore,iav_ong_weight,iav_ong_weightedscore,bws_ong_weight,bws_ong_weightedscore,bwd_ong_weight,bwd_ong_weightedscore,gtd_ong_weight,gtd_ong_weightedscore,sev_ong_weight,sev_ong_weightedscore,rfr_ong_weight,rfr_ong_weightedscore,cfr_ong_weight,cfr_ong_weightedscore,udw_ong_weight,udw_ong_weightedscore,usa_ong_weight,usa_ong_weightedscore,rri_ong_weight,rri_ong_weightedscore,cep_def_weight,cep_def_weightedscore,ucw_def_weight,ucw_def_weightedscore,drr_def_weight,drr_def_weightedscore,iav_def_weight,iav_def_weightedscore,bws_def_weight,bws_def_weightedscore,bwd_def_weight,bwd_def_weightedscore,gtd_def_weight,gtd_def_weightedscore,sev_def_weight,sev_def_weightedscore,rfr_def_weight,rfr_def_weightedscore,cfr_def_weight,cfr_def_weightedscore,udw_def_weight,udw_def_weightedscore,usa_def_weight,usa_def_weightedscore,rri_def_weight,rri_def_weightedscore,cep_fnb_weight,cep_fnb_weightedscore,ucw_fnb_weight,ucw_fnb_weightedscore,drr_fnb_weight,drr_fnb_weightedscore,iav_fnb_weight,iav_fnb_weightedscore,bws_fnb_weight,bws_fnb_weightedscore,bwd_fnb_weight,bwd_fnb_weightedscore,gtd_fnb_weight,gtd_fnb_weightedscore,sev_fnb_weight,sev_fnb_weightedscore,rfr_fnb_weight,rfr_fnb_weightedscore,cfr_fnb_weight,cfr_fnb_weightedscore,udw_fnb_weight,udw_fnb_weightedscore,usa_fnb_weight,usa_fnb_weightedscore,rri_fnb_weight,rri_fnb_weightedscore,cep_che_weight,cep_che_weightedscore,ucw_che_weight,ucw_che_weightedscore,drr_che_weight,drr_che_weightedscore,iav_che_weight,iav_che_weightedscore,bws_che_weight,bws_che_weightedscore,bwd_che_weight,bwd_che_weightedscore,gtd_che_weight,gtd_che_weightedscore,sev_che_weight,sev_che_weightedscore,rfr_che_weight,rfr_che_weightedscore,cfr_che_weight,cfr_che_weightedscore,udw_che_weight,udw_che_weightedscore,usa_che_weight,usa_che_weightedscore,rri_che_weight,rri_che_weightedscore,cep_smc_weight,cep_smc_weightedscore,ucw_smc_weight,ucw_smc_weightedscore,drr_smc_weight,drr_smc_weightedscore,iav_smc_weight,iav_smc_weightedscore,bws_smc_weight,bws_smc_weightedscore,bwd_smc_weight,bwd_smc_weightedscore,gtd_smc_weight,gtd_smc_weightedscore,sev_smc_weight,sev_smc_weightedscore,rfr_smc_weight,rfr_smc_weightedscore,cfr_smc_weight,cfr_smc_weightedscore,udw_smc_weight,udw_smc_weightedscore,usa_smc_weight,usa_smc_weightedscore,rri_smc_weight,rri_smc_weightedscore,cep_agr_weight,cep_agr_weightedscore,ucw_agr_weight,ucw_agr_weightedscore,drr_agr_weight,drr_agr_weightedscore,iav_agr_weight,iav_agr_weightedscore,bws_agr_weight,bws_agr_weightedscore,bwd_agr_weight,bwd_agr_weightedscore,gtd_agr_weight,gtd_agr_weightedscore,sev_agr_weight,sev_agr_weightedscore,rfr_agr_weight,rfr_agr_weightedscore,cfr_agr_weight,cfr_agr_weightedscore,udw_agr_weight,udw_agr_weightedscore,usa_agr_weight,usa_agr_weightedscore,rri_agr_weight,rri_agr_weightedscore,cep_tex_weight,cep_tex_weightedscore,ucw_tex_weight,ucw_tex_weightedscore,drr_tex_weight,drr_tex_weightedscore,iav_tex_weight,iav_tex_weightedscore,bws_tex_weight,bws_tex_weightedscore,bwd_tex_weight,bwd_tex_weightedscore,gtd_tex_weight,gtd_tex_weightedscore,sev_tex_weight,sev_tex_weightedscore,rfr_tex_weight,rfr_tex_weightedscore,cfr_tex_weight,cfr_tex_weightedscore,udw_tex_weight,udw_tex_weightedscore,usa_tex_weight,usa_tex_weightedscore,rri_tex_weight,rri_tex_weightedscore,cep_elp_weight,

In [23]:
destination_table = "{}.{}".format(BQ_OUTPUT_DATASET_NAME,BQ_OUTPUT_TABLE_NAME)

In [24]:
df_merged_weights.to_gbq(destination_table=destination_table,
                         project_id=BQ_PROJECT_ID,
                         chunksize=10000,
                         if_exists="replace")

7it [03:36, 30.93s/it]


In [25]:
destination_path_s3 = "{}/{}.csv".format(ec2_output_path,SCRIPT_NAME)

In [26]:
df_merged_weights.to_csv(destination_path_s3)

In [27]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive

upload: ../../../../data/Y2018M12D11_RH_Master_Weights_GPD_V01/output_V03/Y2018M12D11_RH_Master_Weights_GPD_V01.csv to s3://wri-projects/Aqueduct30/processData/Y2018M12D11_RH_Master_Weights_GPD_V01/output_V03/Y2018M12D11_RH_Master_Weights_GPD_V01.csv


In [28]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:05:22.955155
